<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo5/M5U5_Autoencoders_%26_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1Hh_G3M13P9xSNgSiQ-WnALg93XwK_hG8" alt = "Encabezado MLDS" width = "100%">  </img>

# **_Autoencoders_ y Redes Generativas Adversarias**
---

En este Notebook daremos una breve introducción a métodos generativos de Deep Learning. Los **modelos _generativos_** cubren una clase de modelos estadísticos que contrastan con los **modelos _discriminativos_**. Rápidamente podemos resaltar que :

- Los modelos _generativos_ pueden generar nuevas instancias de datos.
- Los modelos _discriminativos_ discriminan (valga la redundancia) entre distintos tipos de datos.

Es decir, un modelo _generativo_ podría generar nuevas fotos de animales que se parecieran a animales reales, mientras que un modelo discriminativo podría distinguir un perro de un gato.

Más formalmente, dado un conjunto de instancias de datos $X$ y su respectivo conjunto de etiquetas $y$, tenemos que:

- Los modelos _generativos_ capturan la probabilidad conjunta $P(X, y)$, o simplemente $P(X)$ si no hay etiquetas.
- Los modelos _discriminativos_ en cambio capturan la probabilidad condicional $P(y | X)$.

Un **modelo _generativo_** captura la distribución de los propios datos y es capaz de calcular la probabilidad de ocurrencia de un ejemplo determinado. Por ejemplo, los modelos que predicen la siguiente palabra de una secuencia de texto suelen ser modelos generativos porque pueden **asignar una probabilidad** a una secuencia de palabras.

Un **modelo _discriminativo_** en cambio ignora la pregunta de si una instancia dada es probable o no, y simplemente es capaz de **calcular la probabilidad** de que una **etiqueta** se aplique a la instancia.

En este Notebook veremos algunos ejemplos de implementaciones de:
* _Autoencoders_,
* _Autoencoders_ Variacionales,
* Redes Generativas Adversarias (GAN's - _Generative Adversarial Networks_),

Primero importamos _TensorFlow_ y los paquetes que usaremos :

In [ ]:
# Seleccionamos la versión más reciente de Tensorflow 2.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
# Seleccionamos una semilla para los RNG (Random Number Generator)
tf.random.set_seed(123)
np.random.seed(123)
import time

**Tensorflow Probability**

_Tensorflow Probability_ (TFP) es un paquete de _Python_ construido sobre _Tensorflow_ que facilita la combinación de modelos probabilísticos y aprendizaje profundo en hardware moderno (TPU, GPU). TFP incluye:

- Una amplia selección de distribuciones de probabilidad y biyectores.
- Herramientas para construir modelos probabilísticos profundos, incluidas capas probabilísticas y una abstracción `JointDistribution`.
- Métodos de inferencia variacional y _cadena de Markov Monte Carlo_.
- Optimizadores especiales como _Nelder-Mead, BFGS y SGLD_.

Dado que TFP hereda las ventajas de _TensorFlow_, se puede **construir, ajustar y desplegar** un modelo utilizando un único lenguaje.

> **Nota**: Por defecto, TFP no viene instalado en el entorno de Google Colab, así que debemos instalarlo manualmente :

In [ ]:
!pip install tensorflow-probability
!pip install imageio # Paquete para generar gifs
!pip install git+https://github.com/tensorflow/docs

In [ ]:
import os
import PIL
import glob
import imageio.v2 as imageio
import pandas as pd
from IPython import display
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# **1. _Autoencoders_**
---

Los **_Autoencoders_** son un tipo especial de redes neuronales que son típicamente usados en problemas no supervisados y de reducción de dimensionalidad. Se trata de un tipo de arquitecturas que busca beneficiarse del *Representation Learning* o aprendizaje de representación.

En general, con los _Autoencoders_ se busca imponer una restricción de *cuello de botella* con el fin de obtener una representación más compacta de los datos. Un ejemplo típico de la arquitectura de un _Autoencoder_ se puede ver a continuación :

<center><img src="https://drive.google.com/uc?export=view&id=1Kmjwib_nsyLyKYAMt3dBn55VIuj-M--N" alt = "Gráfico ilustrativo de la arquitectura de un Autoencoder " width="60%" /></center>


Veamos algunos componentes generales que conforman un _Autoencoder_ :

* **Encoder** : o _codificador_, se trata de una red neuronal que transforma los datos a un espacio de menor dimensionalidad o _espacio latente_ (en inglés _latent space_, _latent feature space_ o _embedding space_).
* **Decoder** : o _decodificador_, se trata de una red neuronal que transforma los datos del espacio latente al espacio original.

En el ejemplo de la imagen, los datos de entrada tienen 6 dimensiones. El _encoder_ transforma los datos hasta llevarlos a un espacio de 3 dimensiones. Esta nueva representación se llama _Representación Latente_.

- Para garantizar que esta reducción de dimensionalidad conserve la mayor cantidad de información de la representación original, el _decoder_ debe ser capaz de reconstruir los datos originales a partir de la representación latente.

En resumen, un _Autoencoder_ **busca reconstruir  los datos de entrada** utilizando una representación latente. Esto se consigue al optimizar la siguiente función de pérdida :

$$
L=\frac{1}{N}\sum_{i=1}^N ||x_i-\widetilde{x}_i||^2
$$

Donde $x_i$ es una observación original y $\widetilde{x}_i$ está dado por :

$$
\widetilde{x}_i = \text{decoder}(\text{encoder}(x_i)),
$$

y lo que se busca es :
$$
x_i \approx \widetilde{x}_i.
$$
Es decir, el modelo, a través del _encoder_, **transforma los datos originales** a un _espacio latente_, y a partir de ahí debe ser capaz de **reconstruir los datos al espacio original** usando el _decoder_.

Es un concepto muy simple pero su aplicación es bastante útil y posee propiedades bastante interesantes. Veamos un ejemplo de esto en _Tensorflow_.

- Primero vamos a cargar el dataset de imágenes _MNIST_ usando `tf.keras.datasets`:


In [ ]:
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

x_train = train_images.astype('float32') / 255.
x_test = test_images.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)

- Veamos un ejemplo de las imágenes

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(10, 10))
batch = train_images[:11]
for i in range(9):
    ax[i//3, i%3].imshow(np.squeeze(batch[i]))
    ax[i//3, i%3].axis("off")

## **1.1 Autoencoder Básico**
----

Las imágenes con las que vamos a trabajar se representan originalmente en un espacio de dimensión $28\times28=784$. Vamos entonces a definir un _Autoencoder_ con dos capas densas :

1. Un codificador, que comprime las imágenes en un vector latente de 64 dimensiones.
2. Un decodificador, que reconstruye la imagen original a partir de su representación en el espacio latente.

Definamos primero la dimensión del espacio latente :

In [ ]:
latent_dim = 64

Ahora definimos el _Autoencoder_ como una clase a partir de dos modelos secuenciales `tf.keras.Sequential`. Lo vamos a hacer de este modo para que sea más fácil después manipular tanto el modelo completo como el _encoder_ y _decoder_ por separado:

In [ ]:
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential([
                                        layers.Flatten(),
                                        layers.Dense(latent_dim,
                                                     activation='relu'),
                                      ])
    self.decoder = tf.keras.Sequential([
                                        layers.Dense(784,
                                                     activation='sigmoid'),
                                        layers.Reshape((28, 28))
                                      ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

Entendamos el código anterior. El objeto `Autoencoder` recibe como argumento la dimensión latente `latent_dim` y construye el modelo que consta de:

*   El codificador `encoder`, que primero aplana la imagen con `layers.Flatten()` y luego aplica una capa densa que tiene tantas neuronas como se indique en `latent_dim`. Esta capa tiene una activación `relu`.
*   El decodificador `decoder`, que recibe la salida del `encoder` y usa una capa densa de 784 neuronas con activación `sigmoid` para devolver la representación latente al espacio original. La reconstrucción final de la imagen sucede con un `Reshape((28, 28))`.

Con esto en mente podemos definir y nuestro modelo `autoencoder`:

In [ ]:
autoencoder = Autoencoder(latent_dim)

Para compilarlo usamos un optimizador `adam` y, como se había explicado anteriormente, usamos `MeanSquaredError` como función de pérdida. En este caso, el optimizador se define con la tasa de aprendizaje por defecto: `learning_rate=0.001`.

In [ ]:
autoencoder.compile(optimizer='adam',
                    loss=losses.MeanSquaredError())

Entrenamos el modelo utilizando la variable `x_train` tanto como entrada como objetivo.


> El codificador aprenderá a comprimir el conjunto de datos al espacio latente y el decodificador aprenderá a reconstruir las imágenes originales.

In [ ]:
# Entreamos el Autoencoder
autoencoder.fit(x=x_train,
                y=x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

Ahora podemos probar el _Autoencoder_ entrenado, codificando y decodificando imágenes del conjunto de prueba. Primero codifiquemos las imágenes usando el `encoder` del modelo `autoencoder`:

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()

Veamos la dimensión de estas `encoded_imgs`:

In [ ]:
encoded_imgs.shape

Efectivamente tenemos una representación de 64 dimensiones. Ahora decodifiquemos `encoded_imgs` usando el `decoder` :

In [ ]:
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

Y visualicemos algunas de las imágenes originales y su respectiva reconstrucción :

In [ ]:
# Visualizar las imágenes originales y las reconstruidas
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # Mostrar original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # Mostrar reconstrucción
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstruída")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

Este modelo sirve como un reductor de dimensionalidad bastante efectivo:

- Nos permite comprimir la información de cada imagen en vectores de dimensión 64 (casi 12 veces más pequeño que la dimensión original).

**Sin embargo** dependiendo del contexto en el que se esté utilizando el modelo, puede ser importante que las representaciones latentes sigan ciertas propiedades estadísticas que en este caso no se cumplen. Por ejemplo, si se está utilizando un autoencoder para generar datos nuevos a partir de la representación latente, puede ser deseable que las nuevas muestras generadas sigan una distribución específica. En este caso, un autoencoder que aprende representaciones que no siguen una distribución específica puede no ser útil.

Otra posible desventaja de este modelo es que las representaciones latentes pueden ser difíciles de interpretar si no siguen una distribución de probabilidad específica.

- Si se desea comprender cómo el modelo está representando los datos de entrada, puede ser útil tener una idea clara de la distribución de probabilidad subyacente de la representación latente. A continuación vamos a ver cómo mejorar estas limitaciones.




## **1.2. Autoencoder Variacional**
---

El **_Autoencoder_ Variacional** (o _Variational autoencoder_ - VAE) es un modelo generativo basado en _Autoencoders_, el cual busca que el espacio latente represente los **parámetros de una distribución**.

Lo más típico es utilizar una **distribución normal**. Para ello, el espacio latente debe representar sus parámetros, es decir, una media $\mu$ y una desviación típica $\mathbf{\sigma}$ (puede ser un vector asumiendo que no hay correlación en el espacio latente).

<center><img src="https://drive.google.com/uc?export=view&id=1MC8JVeFj1QjGNdxJAAdoSt1koaPeXL19" alt = "Gráfico ilustrativo de un Autoencoder Variacional que utiliza una distribución normal " width="100%" /></center>

Un VAE difiere de un _Autoencoder_ típico en dos aspectos fundamentales:

* 1) La representación latente en un VAE está conformada por **dos capas** (media y desviación típica). El decodificador utiliza entonces una muestra generada con los parámetros estimados por el codificador :

$$
\mu_i, \sigma_i = \text{encoder(x_i)}\\
z_i \sim {N}(\mu_i, \sigma_i)\\
\tilde{x}_i = \text{decoder}(z_i)
$$

* 2) La pérdida se conforma de dos partes :
  - Una pérdida de reconstrucción ${L}_1$ igual a la de los _Autoencoders_ típicos.
  -  Un término de regularización ${L}_2$ el cual busca que las distribuciones generadas por el codificador se parezcan a una distribución normal estándar. Para esto usamos la divergencia de [**_Kullback-Leibler_**](https://es.wikipedia.org/wiki/Divergencia_de_Kullback-Leibler) ($\text{KL}$).

$$
{L}= \lambda_1{L}_1+\lambda_2{L}_2\\
{L}_1 = \text{MSE}(x_i, \tilde{x}_i)\\
{L}_2 = \text{KL}({N}(\mu_i, \sigma_i)|| {N}(0, I))
$$

La divergencia de **_Kullback-Leibler_** es una medida que nos permite ver qué tan parecidas son dos distribuciones. Formalmente, dadas dos distribuciones de variable discreta $P$ y $Q$, la divergencia de Kullback-Leiber está dada por:

$$
D_{KL}=\sum_i P(i)\ln\dfrac{P(i)}{Q(i)}.
$$

En nuestro caso, lo que buscamos que el espacio latente aproxime una distribución normal multivariada con media 0 y desviación 1.


- Comencemos con el ejemplo práctico: Definamos la función `preprocess_images` que escala la imagen en el rango $(0,1)$ (dividiendo por $255$) y binariza completamente la imagen haciendo que los pixeles tomen valor `0.0` o `1.0`:

In [ ]:
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
  return np.where(images > .5, 1.0, 0.0).astype('float32')

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

Definimos variables para la creación de dataset :

In [ ]:
train_size = 60000
batch_size = 32
test_size = 10000

Y usamos `tf.data.Dataset` para crear _Batches_ y mezclar los datos. Recordemos que la API `tf.data.Dataset` permite escribir cadenas de entrada descriptivas y eficientes. El uso de conjuntos de datos sigue un patrón común :

- Crear un conjunto de datos de origen a partir de los datos de entrada.
- Aplicar transformaciones al conjunto de datos para preprocesar los datos.
- Iterar sobre el conjunto de datos y procesar los elementos.

La iteración se produce de **forma continua**, por lo que no es necesario que el conjunto de datos completo quepa en la memoria; es algo parecido a lo que hacíamos con el `ImageDataGenerator` de _Keras_.


In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices(train_images)
                 .shuffle(buffer_size=train_size)
                 .batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(test_images)
                .shuffle(buffer_size=test_size)
                .batch(batch_size))

> **Función `shuffle`**
El código anterior usamos la función `from_tensor_slices` que crea _Batches_ de `train_images` o `test_images` de tamaño `batch_size` después de haber barajado los datos con `shuffle()`. La función `shuffle()` es una operación que reorganiza aleatoriamente los elementos de un dataset. Se utiliza para asegurar que los modelos no aprendan patrones indeseables de los datos debido al orden en que se presentan las muestras durante el entrenamiento.
Cuando se llama a `shuffle(buffer_size)` en un dataset, _TensorFlow_ crea un buffer temporal de tamaño `buffer_size`. A medida que se solicitan elementos del dataset, el siguiente elemento se selecciona aleatoriamente del buffer y se reemplaza con un elemento nuevo del dataset. Esto crea una mezcla aleatoria de los elementos en el dataset.
Un aspecto importante para tener en cuenta es el tamaño del buffer. El tamaño del buffer afecta la aleatoriedad de la mezcla:
* `buffer_size` igual al tamaño del dataset: si el tamaño del buffer es igual al tamaño del dataset, la mezcla será perfectamente aleatoria. Todos los elementos del dataset se cargan en el buffer, y cada elemento tiene igual probabilidad de ser seleccionado en cada iteración. Sin embargo, esto puede tener un impacto en la memoria y el rendimiento, especialmente cuando se trabaja con grandes datasets.
* `buffer_size` menor que el tamaño del dataset: si el tamaño del buffer es menor que el tamaño del dataset, la mezcla será aproximadamente aleatoria. En este caso, se cargan menos elementos en el buffer, lo que reduce el costo computacional y de memoria. Sin embargo, la aleatoriedad será menos precisa, lo que puede afectar la calidad del entrenamiento del modelo.



Ahora definimos un _Autoencoder_. Vamos a hacer algo similar al ejemplo anterior, pero esta vez utilizando convoluciones:

In [ ]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim

    # Definimos el encoder
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                                   filters=32,
                                   kernel_size=3,
                                   strides=(2, 2),
                                   activation='relu'),
            tf.keras.layers.Conv2D(
                                   filters=64,
                                   kernel_size=3,
                                   strides=(2, 2),
                                   activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    # Definimos el encoder
    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32,
                                  activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                                            filters=64,
                                            kernel_size=3,
                                            strides=2,
                                            padding='same',
                                            activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                                            filters=32,
                                            kernel_size=3,
                                            strides=2,
                                            padding='same',
                                            activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                                            filters=1,
                                            kernel_size=3,
                                            strides=1,
                                            padding='same'),
        ]
    )

  # Función para generar una muestra
  @tf.function
  def sample(self, eps=None):
    """
     Esta función se encarga de generar muestras nuevas a partir de un espacio
     latente. Si no se proporciona un tensor eps, se crea uno con forma
     (100, latent_dim) y distribución normal. Luego, pasa eps a la función
     decode() y retorna el resultado.
     """
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  # Función para codificar con el encoder
  def encode(self, x):
    """
     Esta función recibe datos de entrada x y los pasa a través del codificador.
     El codificador devuelve dos valores: la media y la varianza logarítmica
     (logvar) de la distribución latente. La función tf.split() se utiliza para
     dividir el tensor de salida del codificador en dos partes iguales,
     representando la media y la varianza logarítmica.
    """
    mean, logvar = tf.split(self.encoder(x),
                            num_or_size_splits=2,
                            axis=1)
    return mean, logvar

  # Función para hacer un muestreo a partir de una media y varianza
  def reparameterize(self, mean, logvar):
    """
     Esta función toma la media y la varianza logarítmica y realiza el truco de
     la reparametrización. Esto implica muestrear un tensor eps usando una
     distribución normal y luego calcular
     eps * tf.exp(logvar * .5) + mean.
     El resultado es una representación latente que se puede pasar al
     decodificador.
    """
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  # Función para decodificar con el decoder
  def decode(self, z, apply_sigmoid=False):
    """
    Esta función toma la representación latente z y la pasa a través del
    decodificador para generar la reconstrucción de los datos de entrada.
    Si apply_sigmoid es True, se aplica la función sigmoide a los logits del
    decodificador y se devuelve la probabilidad resultante. Si apply_sigmoid es
    False, la función devuelve los logits directamente.
    """
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

**Entendamos el modelo**

De nuevo, tenemos dos bloques importantes, el _encoder_ y el _decoder_:

* _Encoder_: aplica dos capas convolucionales `Conv2D` de 64 y 32 filtros respectivamente. Luego aplica una capa `Flatten` para aplanar la salida de la última convolución pasar el resultado por una capa densa de `latent_dim + latent_dim` neuronas. La primera mitad de este vector de salida corresponde al vector de media $\mu_i$ y la segunda parte al vector de desviaciones  $\sigma_i$.
* _Decoder_: este bloque debe convertir una representación vectorial en una imagen. Es decir, debe hacer una escalada de dimensiones. Para esto usa una capa llamada `Conv2DTranspose`, que cumple la función inversa de `Conv2D`.

### **1.2.1. Convolución transpuesta**
---

También conocida como deconvolución, inversa o transpuesta de una convolución, es una operación matemática que se utiliza para invertir una convolución. La capa `Conv2DTranspose` aplica esta operación a una entrada 2D (generalmente una representación latente) y produce una salida 2D (una imagen generada).

La capa `Conv2DTranspose` es similar a la capa `Conv2D` en su estructura. Tiene varios parámetros que se pueden ajustar para controlar su comportamiento y su rendimiento. A continuación, se describen los parámetros más importantes:

*    `filters`: número entero que representa la cantidad de filtros (también conocidos como kernels) que se utilizan en la capa. Cada filtro es una matriz 2D que se aplica a la entrada para extraer características específicas.

*    `kernel_size`: tamaño de la ventana de convolución en la capa. Es un número entero o una tupla de dos enteros que especifica la altura y el ancho de la ventana de convolución.

*    `strides`: número entero o tupla de dos enteros que especifica el desplazamiento de la ventana de convolución en la dirección horizontal y vertical.

*    `padding`: cadena que puede ser `valid` o `same`. `valid` significa que no se agrega relleno a la entrada y `same` significa que se agrega suficiente relleno para que la salida tenga la misma forma que la entrada.


<center><img src="https://drive.google.com/uc?export=view&id=10P8Oci9m3h04NuiyHhLHJrvYalNBew0i" alt = "Gráfico ilustrativo de un ejemplo de convolución transpuesta "  width="100%" /></center>


### **1.2.2. Función de pérdida**
---

Como siempre, definimos la función de pérdida y el optimizador, teniendo en cuenta lo que explicamos anteriormente sobre aproximar los parámetros de una distribución normal:

In [ ]:
# El optimizador
optimizer = tf.keras.optimizers.Adam(1e-4)

def log_normal_pdf(sample, mean, logvar, raxis=1):
  """
  Esta función calcula la probabilidad logarítmica de una distribución normal
  para un conjunto de muestras sample con una media mean y varianza logarítmica
  logvar. raxis es el eje a lo largo del cual se realiza la suma. La función
  devuelve la suma de las probabilidades logarítmicas de las muestras.
  """
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)

La siguiente función, `compute_loss(model, x)`, calcula la función de pérdida, dada una instancia del modelo y un _batch_ de datos de entrada `x`. La función de pérdida en un VAE consta de dos partes: la divergencia de Kullback-Leibler (KL) entre las distribuciones latentes y la entropía cruzada entre los datos de entrada y su reconstrucción.

   - Primero, se calculan la media y la varianza logarítmica de la distribución latente utilizando la función `encode()`.
   - Luego, se muestrea una representación latente `z` utilizando la función `reparameterize()`.
   - A continuación, se decodifica `z` utilizando la función `decode()` para obtener la reconstrucción `x_logit`.
   - Se calcula la entropía cruzada entre los logits `x_logit` y los datos de entrada `x` usando `tf.nn.sigmoid_cross_entropy_with_logits()`. La suma de esta entropía cruzada a lo largo de los ejes [1, 2, 3] se almacena en `logpx_z`.
   - Se calcula la divergencia KL utilizando la función `log_normal_pdf()` para `logpz` (probabilidad logarítmica de la distribución latente) y `logqz_x` (probabilidad logarítmica de la distribución latente condicional a `x`).
   - Finalmente, se devuelve el negativo de la media de la suma de `logpx_z`, `logpz`, y `-logqz_x`, que es la función de pérdida del VAE.

In [ ]:
def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit,
                                                      labels=x)
  logpx_z = -tf.reduce_sum(cross_ent,
                           axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z,
                           mean,
                           logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)

# Definimos esta función para el entrenamento como lo hacíamos al principio
# del curso usando tf.GradientTape() para tener regístro de todos las etapas
# del entrenamiento

@tf.function
def train_step(model, x, optimizer):
  """
  Esta función calcula la pérdida y los gradientes, y los utiliza para
  actualizar los parámetros del modelo.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

Y a continuación definimos algunos parámetros para el entrenamiento y definimos el modelo. Entrenaremos durante 10 _epochs_ y vamos a fijar la dimensión latente en 2, es decir, en el espacio latente, cada imagen estará representada por dos dimensiones.

In [ ]:
epochs = 10
# Definimos el tamaño del espacio latente de manera que pueda ser visualizado
# posteriormente.
latent_dim = 2
# Definimos el modelo
model = CVAE(latent_dim)

### **1.2.3. *Generación* de muestras**
---

La generación de imágenes con un VAE es muy sencilla, especialmente, porque la pérdida busca que el espacio latente tenga **una distribución normal estándar**. Para obtener vectores válidos en dicho espacio basta con generar un vector de esta distribución.

> Una característica interesante de las representaciones latentes aprendidas con _Variational Autoencoders_ es que las características latentes aprendidas generalmente están decorrelacionadas. Esto es consecuencia de que la función de pérdida busca que la covarianza de las características latentes sea la identidad. Como resultado, las características latentes aprendidas generalmente **codifican un aspecto en particular** de los datos.


Veamos un ejemplo. Definamos primero una función para generar imágenes con el modelo:

In [ ]:
def generate_and_save_images(model, epoch, test_sample):
  mean, logvar = model.encode(test_sample)
  z = model.reparameterize(mean, logvar)
  predictions = model.sample(z)
  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i + 1)
    plt.imshow(predictions[i, :, :, 0],
               cmap='gray')
    plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

Elegimos una muestra del conjunto de prueba para generar imágenes de salida:

In [ ]:
num_examples_to_generate = 16
assert batch_size >= num_examples_to_generate
for test_batch in test_dataset.take(1):
  test_sample = test_batch[0:num_examples_to_generate, :, :, :]

Y generamos las imágenes a medida que entrenamos en modelo:

In [ ]:
generate_and_save_images(model, 0, test_sample)

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(model,
               train_x,
               optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(compute_loss(model,
                      test_x))
  elbo = -loss.result()
  display.clear_output(wait=False)
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))
  generate_and_save_images(model,
                           epoch,
                           test_sample)

Mostramos una imagen generada de la última época de entrenamiento.

In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
plt.imshow(display_image(epoch))
plt.axis('off')


Y mostramos un GIF animado de todas las imágenes guardadas.

In [ ]:
anim_file = 'cvae.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

### **1.2.4. Representación de reducción de dimensionalidad en 2D a partir del espacio latente**
---

Como la dimensión latente del modelo es 2, podemos muestrear datos desde un rectángulo en un plano cartesiano. Lo que se espera es que la variación continua de los parámetros de la distribución debe generar variaciones continuas en las imágenes generadas.

La siguiente función, `plot_latent_images(model, n, digit_size=28)`, sirve para visualizar imágenes generadas por el modelo VAE a partir del espacio latente en una cuadrícula de `n` x `n`. Mas en detalle, la función hace lo siguiente:

1. Crea una distribución normal usando `tfp.distributions.Normal(0, 1)`. Esta distribución se utilizará para generar los puntos en el espacio latente a partir de los cuales se decodificarán las imágenes.

2. Se generan dos secuencias linealmente espaciadas de tamaño `n` entre los cuantiles 0.05 y 0.95 de la distribución normal, llamadas `grid_x` y `grid_y`.

3. Se inicializa una imagen en blanco de tamaño `image_height` x `image_width`, donde ambos son iguales a `digit_size * n`.

4. Se itera sobre los elementos de `grid_x` y `grid_y` y se crea un tensor `z` de dimensión (1, 2) con las coordenadas `(xi, yi)`. Este tensor se pasa a la función `model.sample(z)` para decodificar una imagen a partir del espacio latente.

5. La imagen decodificada se redimensiona a las dimensiones `(digit_size, digit_size)` y se coloca en la posición correspondiente dentro de la imagen en blanco.

6. Una vez que se han generado todas las imágenes y se han colocado en la imagen en blanco, se utiliza `matplotlib` para visualizar la imagen final. Se configura el tamaño de la figura, se muestra la imagen en escala de grises y se ocultan los ejes.

El resultado es una cuadrícula `n` x `n` que muestra las imágenes generadas por el VAE a partir de diferentes puntos en el espacio latente.


In [ ]:
def plot_latent_images(model, n, digit_size=28):
  """Muestra imágenes de n x n de dígitos decodificados del espacio latente."""

  norm = tfp.distributions.Normal(0, 1)
  grid_x = norm.quantile(np.linspace(0.05, 0.95, n))
  grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
  image_width = digit_size*n
  image_height = image_width
  image = np.zeros((image_height, image_width))

  for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
      z = np.array([[xi, yi]])
      x_decoded = model.sample(z)
      digit = tf.reshape(x_decoded[0], (digit_size, digit_size))
      image[i * digit_size: (i + 1) * digit_size,
            j * digit_size: (j + 1) * digit_size] = digit.numpy()

  plt.figure(figsize=(10, 10))
  plt.imshow(image, cmap='Greys_r')
  plt.axis('Off')
  plt.show()

In [ ]:
plot_latent_images(model, 20)

# **2. Redes Generativas Adversarias**
---
Las **redes generativas adversarias** (_Generative Adversarial Networks_ - GAN) son otro tipo de modelos generativos que utilizan redes neuronales. Una GAN rompe el esquema tradicional de _Encoder-Decoder_ y propone dos componentes :

* **Generador**: Se trata de una arquitectura parecida a un _Decoder_, cuya función es transformar vectores aleatorios al espacio de las imágenes.
* **Discriminador**: Se trata de una arquitectura que buscará identificar si una imagen generada es real o falsa.

En otros términos, una GAN se compone de dos modelos que están compitiendo constantemente.

- Por un lado, el generador busca mejorar sus predicciones para engañar al discriminador.
- Por otro, el discriminador busca ser más preciso para detectar qué imágenes son producidas artificialmente en el generador y qué imágenes son reales (del dataset).

<center><img src="https://drive.google.com/uc?export=view&id=12vk0rW6I88WVoPaBq55SOrKRwY3FUJ1u" alt = "Gráfico ilustrativo de una red generativa adversaria " width="100%" /></center>

Veamos el funcionamiento de una GAN. Utilizaremos el dataset MNIST para entrenar el generador y el discriminador. Buscamos que el generador provea dígitos escritos a mano que se parezcan a los pertenecientes a la base de datos MNIST.


In [ ]:
# Cargamos y preparamos los datos
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Generamos un dataset por Batch y combinamos los datos
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## **2.1 Definición del modelo**
---
**Generador**

El generador utiliza capas `tf.keras.layers.Conv2DTranspose` (upsampling) para producir una imagen a partir de una semilla (ruido aleatorio, en este caso de tamaño 100). El generador inicia con una capa densa que tome esta semilla como entrada, luego muestrea varias veces hasta que se alcanza la imagen deseada, en este caso de tamaño 28x28x1.

- Es importante anotar que cada capa utiliza una activación de tipo `tf.keras.layers.LeakyReLU`, excepto por la capa de salida que usa una activación de tipo `tanh`.

La capa `LeakyReLU` es una variante de la función de activación `ReLU` que se caracteriza por permitir valores negativos, lo que puede ayudar a evitar el problema de "neuronas muertas" en las capas convolucionales de la red. Además, `LeakyReLU` introduce un pequeño gradiente negativo en los valores de entrada negativos, lo que puede ayudar a evitar problemas de saturación y mejorar la capacidad del modelo para aprender características sutiles de las muestras.

En modelos tipo GAN, el uso de la capa `LeakyReLU` puede ayudar a mejorar el rendimiento del discriminador al permitir que este aprenda características más sutiles de las muestras reales y generadas, lo que puede hacer que el modelo sea más efectivo para distinguir entre muestras auténticas y generadas.

- Además, `LeakyReLU` puede ayudar a estabilizar el proceso de entrenamiento de la GAN, reduciendo la probabilidad de que el modelo se atasque en un óptimo local o diverja.


In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(shape=(100,)))
    model.add(layers.Dense(7*7*256, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

Podemos observar una imagen obtenida con el generador que aún no ha sido entrenada. Creamos el modelo `generator` y le damos como entrada un vector aleatorio muestreado de una distribución normal:

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

Como era de esperar, la imagen no tiene ningún sentido.

**Discriminador**


Implementamos el discriminador como un clasificador de imágenes basado en CNN. Este modelo debe resolver una tarea binaria: decidir si la imagen que recibe es real o falsa:

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(shape=(28, 28, 1)))
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

Como ejemplo, podemos utilizar el discriminador que aún no ha sido entrenado para clasificar la imagen generada anteriormente. Creamos entonces el modelo `discriminator`. El modelo genera valores positivos para imágenes reales y valores negativos para imágenes falsas.

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

## **2.2 Función de pérdida y optimizador**
---


**Pérdida del discriminador**

Debemos cuantificar qué tan bien el discriminador es capaz de distinguir imágenes reales de falsificaciones. Usamos un `BinaryCrossentropy` que debe comparar las predicciones del discriminador en imágenes reales con una matriz de unos y las predicciones del discriminador en imágenes falsas (generadas) con una matriz de ceros:

In [ ]:
# Cálculo de la pérdida Binary Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

**Pérdida del generador**


La pérdida del generador **cuantifica qué tan bien se pudo engañar al discriminador**. Intuitivamente, si el generador está funcionando bien, el discriminador clasificará las imágenes falsas como reales (1). Para ello se compara la decisión del discriminador sobre las imágenes generadas con una matriz de 1.


In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


El discriminador y los optimizadores del generador son diferentes ya que se entrenarán dos redes por separado. En este caso usamos `Adam` con una tasa de aprendizaje de `1e-4`:

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

**Definición de `Checkpoints`**

Vamos a usar `tf.train.Checkpoint`, que permite guardar los valores de los pesos y otros parámetros del modelo en un momento dado durante el entrenamiento. Esto es útil porque el entrenamiento puede llevar mucho tiempo y se pueden necesitar varias iteraciones para llegar a un modelo entrenado que funcione bien.

- Si el entrenamiento se interrumpe, ya sea de manera intencional o no, se pueden perder los valores de los pesos y otros parámetros que se han ajustado hasta ese momento.

Con `tf.train.Checkpoint`, se pueden guardar los valores de los pesos y otros parámetros de un modelo de manera regular durante el entrenamiento, y restaurarlos más tarde si es necesario. Esto permite reanudar el entrenamiento desde el punto en que se interrumpió o utilizar el modelo entrenado para hacer predicciones en datos nuevos.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

**Definición de ciclo de entrenamiento**

Definimos algunos parámetros necesarios, como el número de `EPOCHS`, la dimensión del vector de ruido que ingresa al generador, y el número de imágenes  generar en cada iteración:

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
# Semilla que será utilizada para poder observar el progreso
seed = tf.random.normal([num_examples_to_generate, noise_dim])

El ciclo de entrenamiento comienza con el generador que recibe una **semilla aleatoria como entrada**. Esa semilla se usa para producir una imagen. Luego, el discriminador se usa para clasificar imágenes reales (extraídas del conjunto de entrenamiento) e imágenes falsas (producidas por el generador).
> La pérdida se calcula para cada uno de estos modelos y los gradientes se utilizan para actualizar el generador y el discriminador.

Definimos entonces la iteración del entrenamiento como una función `train_step`:

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

Y la función `trian` ejecutará el entrenamiento definido en `train_step` según el número de `epochs`. Durante el entrenamiento, además, vamos a guardar los resultados intermedios para visualizar posteriormente el comportamiento del modelo.

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce imágenes para luego construir un GIF
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Guarda el modelo cada 15 epocas
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Genera y guarda imágenes despúes de la última época
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

La siguiente función genera y guarda las imágenes generadas por el modelo:

In [ ]:
# Generar y guardar imágenes
def generate_and_save_images(model, epoch, test_input):
  # `training` es fijado como False para hacer inferencia.

  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

Y entonces podemos entrenar todo.

## **2.3 Entrenamiento del modelo**
---

Con el método `train ()` definido anteriormente es posible entrenar el generador y el discriminador simultáneamente.

> Se debe tener en cuenta que entrenar GAN puede ser complicado. Es importante que el generador y el discriminador **no se dominen entre sí** (por ejemplo, que entrenan a un ritmo similar).

Al comienzo del entrenamiento, las imágenes generadas parecen ruido aleatorio. A medida que avanza el entrenamiento, los dígitos generados se verán cada vez más reales. Después de aproximadamente 50 épocas, se parecerán a los dígitos MNIST.

Entrenemos:

In [ ]:
train(train_dataset, EPOCHS)

Y usemos el `checkpoint` para restaurar los pesos del modelo:

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

**Creamos un GIF**

Como hemos guardado las imágenes que puede generar el modelo después de cada _epoch_, podemos visualizar la evolución de la calidad del proceso generativo.

Primero, veamos lo que puede generar el modelo después de 50 epochs de entrenamiento:

In [ ]:
PIL.Image.open('image_at_epoch_0050.png')


Ahora, utilizamos _imageio_ para crear un gif animado con las imágenes guardadas durante el entrenamiento.

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

Veamos:

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

A nivel generativo las GAN pueden obtener mejores resultados en comparación con los _Autoencoders_. **No obstante**, aunque las GANs son una herramienta poderosa en el aprendizaje profundo, su implementación y entrenamiento pueden ser un desafío debido a algunas dificultades inherentes a la arquitectura de la red y al proceso de entrenamiento.

- **En primer lugar**, las GANs tienen una arquitectura compleja que incluye dos redes neuronales, una generadora y otra discriminadora, que se entrenan de forma adversaria. El entrenamiento de dos redes neuronales al mismo tiempo es más complicado que entrenar una sola red, y esto puede requerir un ajuste más cuidadoso de los hiperparámetros, como la tasa de aprendizaje, el tamaño del lote y el número de capas.

- **En segundo lugar**, las GANs pueden ser difíciles de entrenar debido a la inestabilidad del proceso de entrenamiento. A veces, la red generadora puede aprender a producir muestras que engañan al discriminador, pero estas muestras pueden ser muy diferentes de las muestras reales.

    - Además, el proceso de entrenamiento de las GANs puede ser muy intensivo en términos de recursos computacionales, ya que se requiere entrenar dos redes neuronales al mismo tiempo. Por lo tanto, es posible que se necesite utilizar hardware especializado, como unidades de procesamiento gráfico (GPU) o unidades de procesamiento tensorial (TPU), para acelerar el proceso de entrenamiento.

- **Finalmente**, puede ser difícil evaluar la calidad y la diversidad de las muestras generadas por una GAN, ya que no existe una medida objetiva para la calidad de la generación. Por lo tanto, es posible que se necesite utilizar técnicas adicionales, como el análisis visual o la evaluación humana, para evaluar la calidad y la diversidad de las muestras generadas.

En resumen, la implementación de una GAN puede ser un desafío debido a la complejidad de su arquitectura, la inestabilidad del proceso de entrenamiento, la intensidad computacional del proceso de entrenamiento y la dificultad para evaluar la calidad de las muestras generadas. Sin embargo, a pesar de estas dificultades, las GANs siguen siendo una herramienta poderosa en el aprendizaje profundo y se utilizan en una amplia variedad de aplicaciones, desde la generación de imágenes hasta la creación de música y el diseño de moda.

# **Recursos adicionales**
----

- [*Introducción a los codificadores automáticos*](https://www.tensorflow.org/tutorials/generative/autoencoder?hl=es-419)

- [*Codificador automático variacional convolucional*](https://www.tensorflow.org/tutorials/generative/cvae?hl=es-419)

# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
  * [Juan Sebastián Lara](https://http://juselara.com/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](https://www.linkedin.com/in/mario-andres-rodriguez-triana-394806145/).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*